This notebook is for those who cannot import cifar10 dataset directly from Tensorflow Keras Dataset. 

Firstly, please downlod the data from the below website. 
https://www.cs.toronto.edu/~kriz/cifar.html

Once downloaded, please place the file in where this notebook is. 

Please run the below cells.

In [1]:
#libraries
import numpy as np

# for unzipping the downloaded file
import tarfile


In [2]:
# unzipping the file.
import tarfile

fname = "cifar-10-python.tar.gz"

if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

In [3]:
# function to read files
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
# training set
for i in range(5):
    fname = f'cifar-10-batches-py/data_batch_{i+1}'

    data = unpickle(fname)
    
    X = data[b'data']
    y = data[b'labels']
    y = np.array(y)
    
    exec(f'X_train_data_{i+1} = X' )
    exec(f'y_train_data_{i+1} = y' )
    
# test set
data = unpickle('cifar-10-batches-py/test_batch')
X_test = data[b'data']
y_test = np.array(data[b'labels'])


In [5]:
# combining all the data
X_train_data_all = np.concatenate((X_train_data_1, 
                                   X_train_data_2, 
                                   X_train_data_3, 
                                   X_train_data_4, 
                                   X_train_data_5), axis=0)
y_train_data_all = np.concatenate((y_train_data_1, 
                                   y_train_data_2, 
                                   y_train_data_3, 
                                   y_train_data_4, 
                                   y_train_data_5), axis=0)

print("X train data shape: ", X_train_data_all.shape)
print("y train data shape: ", y_train_data_all.shape)


X train data shape:  (50000, 3072)
y train data shape:  (50000,)


In [6]:
# Important!!! 
# reshaping the data into 32x32 with 3 layers for red, green and blue (RGB)
X_train = X_train_data_all.reshape(X_train_data_all.shape[0], 3, 32, 32).transpose(0,2,3,1).astype("uint8")

X_test = X_test.reshape(X_test.shape[0], 3, 32, 32).transpose(0,2,3,1)

# setting y_train and y_test
y_train = y_train_data_all


print("X train data shape: ", X_train.shape)
print("y train data shape: ", X_test.shape)


X train data shape:  (50000, 32, 32, 3)
y train data shape:  (10000, 32, 32, 3)


# Libraries for model building

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix


# import tensorflow and kera
import tensorflow
from tensorflow import keras
from keras.datasets import cifar10
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential
from keras.utils import to_categorical

from keras.callbacks import TensorBoard


In [8]:
# Prepare input data
_, img_rows, img_cols, img_channels = X_train.shape
num_classes = len(set(y_train.flatten()))

# Convert the target to categorical
y_train = to_categorical(
    y_train,
    num_classes = num_classes,
    dtype = 'uint8')
y_test = to_categorical(
    y_test,
    num_classes = num_classes,
    dtype = 'uint8')

# Model building

In [9]:
model = Sequential()

In [10]:
model.add(Conv2D(48, kernel_size = 3, activation = 'relu', padding = 'same', input_shape = (32, 32, 3)))
model.add(Conv2D(48, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(96, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(Conv2D(96, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(192, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(Conv2D(192, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 48)        1344      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 48)        20784     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 48)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 96)        41568     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 96)        83040     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 96)          0

In [12]:
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [13]:
tensorboard = TensorBoard(log_dir = 'logs')

In [ ]:
%%time
# Fit the model on a training set
history = model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    # epochs = 100,
    epochs = 100,
    # batch_size = 100,
    batch_size = 100,
    callbacks = [tensorboard],
    verbose = 1)
print(f'Training accuracy:{history.history["accuracy"][-1]:.2f} validation accuracy:{history.history["val_accuracy"][-1]:.2f} ')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
%%time
predictions = model.predict_classes(X_test)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size = 10)
print('\nTest loss: %.6f, Test accuracy: %.6f' % tuple(score))